In [1]:
import arcpy
import os
import re
 
def sanitize_name(name):
    """
    Sanitizes a string to create a valid feature class name.
    Removes invalid characters and replaces spaces with underscores.
    Parameters:
        name (str): The input name.
    Returns:
        str: The sanitized name.
    """
    # Replace invalid characters with underscores and remove remaining invalid characters
    name = re.sub(r"[^a-zA-Z0-9_]", "_", name)
    return name[:64]  # Limit to 64 characters for geodatabase naming
 
def process_excel_to_featureclass(input_folder, output_gdb, merge_gdb, x_field='X', y_field='Y'):
    """
    Converts Excel files in a folder to feature classes in a file geodatabase and merges them.
    Parameters:
        input_folder (str): Path to the folder containing Excel files.
        output_gdb (str): Path to the file geodatabase for output feature classes.
        merge_gdb (str): Path to the file geodatabase where merged output will be stored.
        x_field (str): Field name for X coordinates. Default is 'X'.
        y_field (str): Field name for Y coordinates. Default is 'Y'.
    """
    try:
        # Define spatial reference for WGS_1984_UTM_38N
        spatial_ref = arcpy.SpatialReference(32638)  # EPSG code for WGS_1984_UTM_38N
 
        # Ensure the output GDB exists, overwrite if necessary
        if arcpy.Exists(output_gdb):
            arcpy.management.Delete(output_gdb)
        arcpy.management.CreateFileGDB(os.path.dirname(output_gdb), os.path.basename(output_gdb))
 
        # Create a list to hold all feature classes for merging
        feature_classes = []
 
        # Process each Excel file in the folder
        for file in os.listdir(input_folder):
            if file.endswith('.xlsx') or file.endswith('.xls'):
                excel_path = os.path.join(input_folder, file)
                table = f"{excel_path}\\Sheet1$"  # Assumes data is in Sheet1
                # Make XY Event Layer
                event_layer = "temp_event_layer"
                arcpy.management.MakeXYEventLayer(
                    table, x_field, y_field, event_layer,
                    spatial_ref
                )
                # Export to Feature Class
                fc_name = sanitize_name(os.path.splitext(file)[0])  # Sanitize name
                fc_path = os.path.join(output_gdb, fc_name)
                arcpy.management.CopyFeatures(event_layer, fc_path)
                feature_classes.append(fc_path)
                print(f"Processed: {file} -> {fc_path}")
 
        # Merge all feature classes into a single output in the merge GDB
        merged_fc = os.path.join(merge_gdb, "MergedFeatures")
        if feature_classes:
            arcpy.management.Merge(feature_classes, merged_fc)
            print(f"Merged feature classes into: {merged_fc}")
        else:
            print("No valid Excel files found in the input folder.")
 
    except Exception as e:
        print(f"Error: {e}")
 
# Define input parameters
input_folder = r"C:\Users\skannan\excel\2025"  # Replace with your folder path
output_gdb = r"C:\Users\skannan\GDB\2025\gdb\output.gdb"  # Replace with your output GDB path
merge_gdb = r"C:\Users\skannan\GDB\2025\gdb\merge.gdb"    # Replace with your merge GDB path
 
# Call the function
process_excel_to_featureclass(input_folder, output_gdb, merge_gdb)

Processed: Crew1.xlsx -> C:\Users\skannan\2025\gdb\output.gdb\Crew1
Processed: Crew3.xlsx -> C:\Users\skannan\2025\gdb\output.gdb\Crew3
Processed: Crew4.xlsx -> C:\Users\skannan\2025\gdb\output.gdb\Crew4
Merged feature classes into: C:\Users\skannan\2025\gdb\merge.gdb\MergedFeatures
